# Velocity Data Download Script for Validation
#### Working Script to pull large velocity datasets and later compare to ICESAT2-derived velocities

ICESat-2 hackweek  
June 15, 2020  
Lynn Kaluzienski

# Import necessary modules

In [ ]:
import os,re,h5py
import requests
import zipfile

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import scipy, sys, os, pyproj, glob
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
#! cd ..; [ -d pointCollection ] || git clone https://www.github.com/smithB/pointCollection.git
#sys.path.append(os.path.join(os.getcwd(), '..'))

import pointCollection as pc
#stuck here, not sure how to import point collection 


In [ ]:
#Magic function to enable interactive plotting (zoom/pan) in Jupyter notebook
#If running locally, this would be `%matplotlib notebook`, but since we're using Juptyerlab, we use widget
%matplotlib widget
#%matplotlib inline

In [ ]:
'''
##Section where I played around with loading in .mat file, quickly abandoned
#load in data from .mat file
#Data originally in netcdf format, created subset in matlab for FIS region and saved as .mat file. 
import scipy.io as spio
data_root='/srv/surface_velocity/contributors/lynn_kaluzienski/Data_test'
#('Measures2.mat',squeeze_me=True)
Measures2_Files= spio.loadmat('./Data_test/Measures2.mat')
x=Measures2_Files['posx']
y=Measures2_Files['posy']
vx=Measures2_Files['vx_FIS']
vy=Measures2_Files['vy_FIS']
vel=Measures2_Files['vel_FIS']

#xy=np.array(pyproj.Proj(3031)(x, y))
plt.figure()
ax.imshow(vel, extent=[0, 1, 0, 1])
''';

In [ ]:
#From Ben Smith's code loading in .tif file, running into issues likely with directories
data_root='/srv/shared/surface_velocity/FIS_Velocity/'
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
#Originally tried to load data from a local directory, should change to shared directory
Measures_vel=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vel.tif'), bounds=[XR, YR])

# show the velocity map:
plt.figure()
Measures_vel.show(cmap='viridis', clim=[0,100])
plt.plot(xy[0,:], xy[1,:])
plt.title('Measures2 Velocity Map')


In [ ]:
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

In [ ]:
# Load a line and plot

data_root='/srv/shared/surface_velocity/'
field_dict={None:['delta_time','latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}

rgt = "0848"
cycle="03"
filename == glob.glob(os.path.join(data_root, 'FIS_ATL06', f'*ATL06_*_{rgt}{cycle}*_003*.h5'))

D=atl06_to_dict(filename,'/gt2l', field_dict=field_dict, index=None, epsg=3031)

# show the velocity map:
plt.figure()
Measures_vel.show(cmap='viridis', clim=[0,400])
plt.plot(xy[0,:], xy[1,:],'k')
plt.title('Measures2 Velocity Map')

plt.plot(D['x'],D['y'],'r')

In [ ]:
# Interpolate the Measures velocities along the line and plot (note that these are speeds for now)

V = Measures_vel.interp(D['x'],D['y'])

plt.figure()
plt.plot(D['x_atc'],V)